In [2]:
! pip install opencv-python -q


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import numpy as np
import json
import os

def create_output_directories(base_dir, test_number):
    test_dir = os.path.join(base_dir, f"test_{test_number}")
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    return test_dir

def save_images(image, image_name, test_dir):
    image_path = os.path.join(test_dir, image_name)
    cv2.imwrite(image_path, image)

def count_glowing_leds(gray_image, threshold=200, debug=False, test_dir=None, frame_number=0):
    # Simple threshold to identify glowing LEDs
    _, binary_image = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY)

    # Find contours of the glowing areas
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if debug:
        # Save and display grayscale image
        gray_image_name = f"frame_{frame_number}_grayscale.png"
        save_images(gray_image, gray_image_name, test_dir)
        # cv2.imshow('Grayscale Image', gray_image)

        # Save and display binary image
        binary_image_name = f"frame_{frame_number}_binary.png"
        save_images(binary_image, binary_image_name, test_dir)
        # cv2.imshow('Binary Image', binary_image)

        # Draw contours on the grayscale image and save/display it
        contour_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
        contour_image_name = f"frame_{frame_number}_contours.png"
        save_images(contour_image, contour_image_name, test_dir)
        # cv2.imshow('Contour Image', contour_image)

        # Wait for user input to close the image windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return len(contours)

def process_video(video_path, samples_per_minute=24, threshold=200, debug=False):
    # Create base output directory
    base_dir = "outputs"
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # Determine test number by counting existing tests
    existing_tests = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    test_number = len(existing_tests) + 1

    # Create directory for this test run
    test_dir = create_output_directories(base_dir, test_number)

    # Load the video
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / frame_rate

    # Calculate the interval between frames to sample
    sample_interval = int(frame_rate * (60 / samples_per_minute))

    results = []
    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_number % sample_interval == 0:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            led_count = count_glowing_leds(gray_frame, threshold, debug, test_dir, frame_number)
            results.append({
                'frame': frame_number,
                'leds_glowing': led_count
            })

        frame_number += 1

    cap.release()

    # Save results to a JSON file
    results_path = os.path.join(test_dir, 'leds_glowing_results.json')
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4)

    return results

# Example usage with debugging enabled
video_path = 'video/sample_2.avi'
samples_per_minute = 24  # Change this value as needed
threshold = 240  # Adjust the threshold based on the brightness of LEDs (default 200)
process_video(video_path, samples_per_minute, threshold, debug=True)
